### Process txt

In [ ]:
import os
import pandas as pd
import cv2
import re
import glob

def read_anotation(file_path):
    annotations = []
    with open(file_path, 'r', encoding="utf-8") as lines:
        for line in lines:
            path = line.strip().split(",")
            x1,y1,x2,y2,x3,y3,x4,y4,text = int(path[0]), int(path[1]), int(path[2]), int(path[3]),  int(path[4]), int(path[5]), int(path[6]), int(path[7]), path[8]
            annotations.append([x1,y1,x3,y3,text])
    return annotations

def write_anotation(file, annotations, input_path, count):
    for i,annotation in enumerate(annotations):
        *_,text = annotation
        new_input_path = input_path[:-4] + f"_{i}.txt"
        line = f"{new_input_path}\t{text}\n"
        file.write(line)
    return count + i

def img_crop(annotations, txt_path, new_dir):
    img_path = txt_path[:-4] + f".jpg"
    img_name = os.path.basename(txt_path).split(".")[0]
    for i,annotation in enumerate(annotations):
        new_img_path = new_dir + img_name + f"_{i}.jpg"
        x1,y1,x3,y3,text = annotation
        img = cv2.imread(img_path)
        croped_img = img[y1:y3, x1:x3]
        cv2.imwrite(new_img_path, croped_img)


def preprocess(current_dir, new_dir, output_file):
    with open (output_file, "w", encoding="utf-8") as out_f:
        count = 0
        for txt_path in glob.glob(os.path.join(current_dir, "*.txt")):
            img_path = txt_path[:-4] + f".jpg"
            if "(" in txt_path or cv2.imread(img_path) is None:
                print(f"Ignore:{img_path}")
                continue
            print(img_path)
            annotations = read_anotation(txt_path)
            count = write_anotation(out_f, annotations, txt_path, count)
            img_crop(annotations, txt_path, new_dir)
            print(count)

current_dir = "./invoice_ocr/img/"
new_dir = "./invoice_ocr/img_crop/"
train_txt = "./invoice_ocr/invoice_label_train.txt"
preprocess(current_dir, new_dir, train_txt)

In [ ]:
def split_txt(input_file, output_file, n_lines):
    with open(input_file, 'r', encoding="utf-8" ) as infile:
        lines = infile.readlines()

    with open(output_file, 'w', encoding="utf-8") as outfile:
        outfile.writelines(lines[:n_lines])
    with open(input_file, 'w', encoding="utf-8" ) as infile:
        infile.writelines(lines[n_lines:])

train_txt = "./invoice_ocr/invoice_label_train.txt"
test_txt = "./invoice_ocr/invoice_label_test.txt"
split_txt(train_txt,test_txt, 11000)